In [95]:
import pandas as pd
df = pd.read_parquet("data/data_vogue_final.parquet")
df = df[df.groupby("fashion_house")["fashion_house"].transform("count") >= 20]
#df = df.drop(columns=[col for col in ["designer_name"] if col in df.columns])

problem_rows = df[df["designer_name"].apply(lambda x: len(x) ==0)]

In [96]:
df

,fashion_house,show,URL,cover_image_url,year,category,season,location,description,editor,publish_date,designer_name,image_urls,brand_name,country,city,website,foundation_year,founder,belongs_to
0,Aalto,fall-2016-ready-to-wear,https://www.vogue.com/fashion-shows/fall-2016-...,https://assets.vogue.com/photos/56d9a95440a5ec...,2016,ready-to-wear,fall,,People were not supposed to sit at the Aalto s...,Alessandra Codinha,"March 4, 2016",Tuomas Merikoski,[https://assets.vogue.com/photos/56d9a95440a5e...,Aalto,France,Paris,https://www.aaltointernational.com/,2015.0,Tuomas Merikoski,None
1,Aalto,fall-2017-ready-to-wear,https://www.vogue.com/fashion-shows/fall-2017-...,https://assets.vogue.com/photos/58b5a53696d4a6...,2017,ready-to-wear,fall,,"In some respects, this latest Aalto collection...",Amy Verner,"February 28, 2017",Tuomas Merikoski,[https://assets.vogue.com/photos/58b5a53696d4a...,Aalto,France,Paris,https://www.aaltointernational.com/,2015.0,Tuomas Merikoski,None
2,Aalto,pre-fall-2017,https://www.vogue.com/fashion-shows/pre-fall-2...,https://assets.vogue.com/photos/5879422af1829e...,2017,,pre-fall,,AssessingAaltocollections strictly by their ti...,Amy Verner,"January 13, 2017",Tuomas Merikoski,[https://assets.vogue.com/photos/5879422af1829...,Aalto,France,Paris,https://www.aaltointernational.com/,2015.0,Tuomas Merikoski,None
3,Aalto,spring-2017-ready-to-wear,https://www.vogue.com/fashion-shows/spring-201...,https://assets.vogue.com/photos/57ea8315734e55...,2017,ready-to-wear,spring,,Many a fashion mood board has boasted pictures...,Alessandra Codinha,"September 27, 2016",Tuomas Merikoski,[https://assets.vogue.com/photos/57ea8315734e5...,Aalto,France,Paris,https://www.aaltointernational.com/,2015.0,Tuomas Merikoski,None
4,Aalto,resort-2017,https://www.vogue.com/fashion-shows/resort-201...,https://assets.vogue.com/photos/57695f8441a300...,2017,,resort,,Aaltois one of two brands that made this year’...,Amy Verner,"June 21, 2016",Tuomas Merikoski,[https://assets.vogue.com/photos/57695f8441a30...,Aalto,France,Paris,https://www.aaltointernational.com/,2015.0,Tuomas Merikoski,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18908,Zuhair Murad,resort-2024,https://www.vogue.com/fashion-shows/resort-202...,https://assets.vogue.com/photos/649aa1e5c1aeb0...,2024,,resort,,Zuhair Murad has stars in his eyes. One is Tay...,Tina Isaac-Goizé,"June 28, 2023",Zuhair Murad,[https://assets.vogue.com/photos/649aa1e5c1aeb...,Zuhair Murad,France,Paris,https://www.zuhairmurad.com,1995.0,Murad,Murad Zuhair
18909,Zuhair Murad,spring-2025-ready-to-wear,https://www.vogue.com/fashion-shows/spring-202...,https://assets.vogue.com/photos/66fac981075851...,2025,ready-to-wear,spring,,The sea is shaping up to be one of spring’s ma...,Tina Isaac-Goizé,"October 2, 2024",Zuhair Murad,[https://assets.vogue.com/photos/66fac98107585...,Zuhair Murad,France,Paris,https://www.zuhairmurad.com,1995.0,Murad,Murad Zuhair
18910,Zuhair Murad,spring-2025-ready-to-wear,https://www.vogue.com/fashion-shows/spring-202...,https://assets.vogue.com/photos/66fac981075851...,2025,ready-to-wear,spring,,The sea is shaping up to be one of spring’s ma...,Tina Isaac-Goizé,"October 2, 2024",Zuhair Murad,[https://assets.vogue.com/photos/66fac98107585...,Zuhair Murad,France,Paris,https://www.zuhairmurad.com,1995.0,Murad,Murad Zuhair
18911,Zuhair Murad,resort-2025,https://www.vogue.com/fashion-shows/resort-202...,https://assets.vogue.com/photos/6672a4d49ee2d4...,2025,,resort,,Whether she’s a planner or a last-minute dress...,Tina Isaac-Goizé,"June 17, 2024",Zuhair Murad,[https://assets.vogue.com/photos/6672a4d49ee2d...,Zuhair Murad,France,Paris,https://www.zuhairmurad.com,1995.0,Murad,Murad Zuhair


In [97]:
df_exp = df.explode("image_urls")
df_exp

,fashion_house,show,URL,cover_image_url,year,category,season,location,description,editor,publish_date,designer_name,image_urls,brand_name,country,city,website,foundation_year,founder,belongs_to
0,Aalto,fall-2016-ready-to-wear,https://www.vogue.com/fashion-shows/fall-2016-...,https://assets.vogue.com/photos/56d9a95440a5ec...,2016,ready-to-wear,fall,,People were not supposed to sit at the Aalto s...,Alessandra Codinha,"March 4, 2016",Tuomas Merikoski,https://assets.vogue.com/photos/56d9a95440a5ec...,Aalto,France,Paris,https://www.aaltointernational.com/,2015.0,Tuomas Merikoski,None
0,Aalto,fall-2016-ready-to-wear,https://www.vogue.com/fashion-shows/fall-2016-...,https://assets.vogue.com/photos/56d9a95440a5ec...,2016,ready-to-wear,fall,,People were not supposed to sit at the Aalto s...,Alessandra Codinha,"March 4, 2016",Tuomas Merikoski,https://assets.vogue.com/photos/56d9a954d44ad6...,Aalto,France,Paris,https://www.aaltointernational.com/,2015.0,Tuomas Merikoski,None
0,Aalto,fall-2016-ready-to-wear,https://www.vogue.com/fashion-shows/fall-2016-...,https://assets.vogue.com/photos/56d9a95440a5ec...,2016,ready-to-wear,fall,,People were not supposed to sit at the Aalto s...,Alessandra Codinha,"March 4, 2016",Tuomas Merikoski,https://assets.vogue.com/photos/56d9a9572a88e6...,Aalto,France,Paris,https://www.aaltointernational.com/,2015.0,Tuomas Merikoski,None
0,Aalto,fall-2016-ready-to-wear,https://www.vogue.com/fashion-shows/fall-2016-...,https://assets.vogue.com/photos/56d9a95440a5ec...,2016,ready-to-wear,fall,,People were not supposed to sit at the Aalto s...,Alessandra Codinha,"March 4, 2016",Tuomas Merikoski,https://assets.vogue.com/photos/56d9a9572e5208...,Aalto,France,Paris,https://www.aaltointernational.com/,2015.0,Tuomas Merikoski,None
0,Aalto,fall-2016-ready-to-wear,https://www.vogue.com/fashion-shows/fall-2016-...,https://assets.vogue.com/photos/56d9a95440a5ec...,2016,ready-to-wear,fall,,People were not supposed to sit at the Aalto s...,Alessandra Codinha,"March 4, 2016",Tuomas Merikoski,https://assets.vogue.com/photos/56d9a9589fd7a9...,Aalto,France,Paris,https://www.aaltointernational.com/,2015.0,Tuomas Merikoski,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18912,Zuhair Murad,resort-2025,https://www.vogue.com/fashion-shows/resort-202...,https://assets.vogue.com/photos/6672a4d49ee2d4...,2025,,resort,,Whether she’s a planner or a last-minute dress...,Tina Isaac-Goizé,"June 17, 2024",Zuhair Murad,https://assets.vogue.com/photos/66705389a106e0...,Zuhair Murad,France,Paris,https://www.zuhairmurad.com,1995.0,Murad,Murad Zuhair
18912,Zuhair Murad,resort-2025,https://www.vogue.com/fashion-shows/resort-202...,https://assets.vogue.com/photos/6672a4d49ee2d4...,2025,,resort,,Whether she’s a planner or a last-minute dress...,Tina Isaac-Goizé,"June 17, 2024",Zuhair Murad,https://assets.vogue.com/photos/6670538b8fb082...,Zuhair Murad,France,Paris,https://www.zuhairmurad.com,1995.0,Murad,Murad Zuhair
18912,Zuhair Murad,resort-2025,https://www.vogue.com/fashion-shows/resort-202...,https://assets.vogue.com/photos/6672a4d49ee2d4...,2025,,resort,,Whether she’s a planner or a last-minute dress...,Tina Isaac-Goizé,"June 17, 2024",Zuhair Murad,https://assets.vogue.com/photos/6670539047fdff...,Zuhair Murad,France,Paris,https://www.zuhairmurad.com,1995.0,Murad,Murad Zuhair
18912,Zuhair Murad,resort-2025,https://www.vogue.com/fashion-shows/resort-202...,https://assets.vogue.com/photos/6672a4d49ee2d4...,2025,,resort,,Whether she’s a planner or a last-minute dress...,Tina Isaac-Goizé,"June 17, 2024",Zuhair Murad,https://assets.vogue.com/photos/66705390075f00...,Zuhair Murad,France,Paris,https://www.zuhairmurad.com,1995.0,Murad,Murad Zuhair


In [88]:
df_designers = pd.read_parquet("data/final_info_designers.parquet")
df_designers

,designer_name,place_of_birth,year_birth,education,employer
0,Aage Thaarup,Copenhagen,1906.0,None,[]
1,Achille Maramotti,Reggio Emilia,1850.0,Sapienza University of Rome,[Max Mara]
2,Achille Maramotti,Reggio Emilia,1927.0,None,None
3,Ada Masotti,Italy,NaN,None,[La Perla]
4,Adam Kimmel,New York,2002.0,None,[Adam Kimmel]
...,...,...,...,...,...
678,Zandra Rhodes,Chatham,1940.0,Kent Institute of Art & Design,None
679,Zandra Rhodes,Kent,1961.0,None,[]
680,Zoe Latta,None,NaN,Rhode Island School of Design,None
681,Zowie Broach,None,NaN,None,None


In [3]:
df_exp = df.explode("designer_name")

# Remove None or empty strings if any
df_exp = df_exp[df_exp["designer_name"].notna() & (df_exp["designer_name"] != "")]

from collections import Counter

counts = Counter(df_exp.designer_name)
designers20= {name for name, cnt in counts.items() if cnt >= 20}
len(designers20)

406

In [32]:
df[df["designer_name"]=="Delphine Ninous"]

,fashion_house,show,URL,cover_image_url,year,category,season,location,description,editor,publish_date,designer_name,brand_name,country,city,website,foundation_year,founder
2598,Belstaff,fall-2016-ready-to-wear,https://www.vogue.com/fashion-shows/fall-2016-...,https://assets.vogue.com/photos/56ca0343001a07...,2016,ready-to-wear,fall,,There must be something in the water atBelstaf...,Luke Leitch,"February 21, 2016",Delphine Ninous,Belstaff,Italy,Milan,https://www.belstaff.com,1924.0,Eli Belovitch
2600,Belstaff,pre-fall-2016,https://www.vogue.com/fashion-shows/pre-fall-2...,https://assets.vogue.com/photos/564e343b77a8a3...,2016,,pre-fall,,"Pre-Fallalready? Golly, that was fast. Belstaf...",Luke Leitch,"November 19, 2015",Delphine Ninous,Belstaff,Italy,Milan,https://www.belstaff.com,1924.0,Eli Belovitch
2601,Belstaff,spring-2016-ready-to-wear,https://www.vogue.com/fashion-shows/spring-201...,https://assets.vogue.com/photos/5600298a79d61b...,2016,ready-to-wear,spring,,How better to purge any lingering associationa...,Luke Leitch,"September 21, 2015",Delphine Ninous,Belstaff,Italy,Milan,https://www.belstaff.com,1924.0,Eli Belovitch
2602,Belstaff,resort-2016,https://www.vogue.com/fashion-shows/resort-201...,https://assets.vogue.com/photos/55c650be08298d...,2016,,resort,,Easy Rider? The Belstaff woman used to look mo...,Luke Leitch,"June 24, 2015",Delphine Ninous,Belstaff,Italy,Milan,https://www.belstaff.com,1924.0,Eli Belovitch
2604,Belstaff,fall-2017-menswear,https://www.vogue.com/fashion-shows/fall-2017-...,https://assets.vogue.com/photos/58ad9aa7692b7b...,2017,menswear,fall,,Now the sole captain charged with steering des...,Luke Leitch,"February 22, 2017",Delphine Ninous,Belstaff,Italy,Milan,https://www.belstaff.com,1924.0,Eli Belovitch
2605,Belstaff,fall-2017-ready-to-wear,https://www.vogue.com/fashion-shows/fall-2017-...,https://assets.vogue.com/photos/58a72a95be5c61...,2017,ready-to-wear,fall,,Now the sole captain charged with steering des...,Luke Leitch,"February 17, 2017",Delphine Ninous,Belstaff,Italy,Milan,https://www.belstaff.com,1924.0,Eli Belovitch
2606,Belstaff,spring-2017-ready-to-wear,https://www.vogue.com/fashion-shows/spring-201...,https://assets.vogue.com/photos/57def54422da51...,2017,ready-to-wear,spring,,"Paging Baz Luhrmann, Ron Howard, or even the b...",Luke Leitch,"September 18, 2016",Delphine Ninous,Belstaff,Italy,Milan,https://www.belstaff.com,1924.0,Eli Belovitch
2607,Belstaff,resort-2017,https://www.vogue.com/fashion-shows/resort-201...,https://assets.vogue.com/photos/575da022ddbe18...,2017,,resort,,"This season(s), the motorcycling heritage bran...",Luke Leitch,"June 12, 2016",Delphine Ninous,Belstaff,Italy,Milan,https://www.belstaff.com,1924.0,Eli Belovitch
2608,Belstaff,fall-2018-ready-to-wear,https://www.vogue.com/fashion-shows/fall-2018-...,https://assets.vogue.com/photos/5a8b772171dfb1...,2018,ready-to-wear,fall,,Delphine Ninous has proved a highly able creat...,Luke Leitch,"February 19, 2018",Delphine Ninous,Belstaff,Italy,Milan,https://www.belstaff.com,1924.0,Eli Belovitch
2609,Belstaff,fall-2018-ready-to-wear,https://www.vogue.com/fashion-shows/fall-2018-...,https://assets.vogue.com/photos/5a8b772171dfb1...,2018,ready-to-wear,fall,,Delphine Ninous has proved a highly able creat...,Luke Leitch,"February 19, 2018",Delphine Ninous,Belstaff,Italy,Milan,https://www.belstaff.com,1924.0,Eli Belovitch


In [30]:
df[
    #(df["fashion_house"] == "Belstaff") 
    # &
     (df["designer_name"].apply(lambda x: "Alexander Mcqueen" in list(x)))
]

,fashion_house,show,URL,cover_image_url,year,category,season,location,description,editor,publish_date,designer_name,brand_name,country,city,website,foundation_year,founder


In [92]:
import os
device_dir = os.getcwd().split('/')[2]

df_emb = pd.read_json(f"/Users/{device_dir}/Library/CloudStorage/OneDrive-UvA/fashion_images/fashion_show_data_all_embeddings_allpics_segmented_white_pivoted.json", lines=True)


In [107]:
import numpy as np
import pandas as pd


import numpy as np

def df_to_npy_clean(df, image_prefix='image_url', clip_prefix='fashion_clip', n_images=11, save_path='data/fashion_data_emb.npy'):
    data = []

    for i in range(1, n_images + 1):
        image_col = f'{image_prefix}{i}'
        clip_col = f'{clip_prefix}{i}'
        
        valid = df[image_col].notna() & df[clip_col].notna()
        for img, clip in zip(df.loc[valid, image_col], df.loc[valid, clip_col]):
            # Convert list to numpy array
            clip_array = np.array(clip, dtype=np.float32)
            data.append((img, clip_array))
    
    # Save as object array (tuples of image + embedding)
    data_array = np.array(data, dtype=object)
    np.save(save_path, data_array)
    return data_array

# Usage
data_array = df_to_npy_clean(df_emb)
print(data_array.shape)
print(data_array[:5])

(136249, 2)
[['https://assets.vogue.com/photos/591b5547ecc6916578b6f84b/master/w_1280,c_limit/_SOR0298.jpg'
  array([-5.24466485e-02,  1.27326041e-01, -2.74838544e-02, -1.23123433e-02,
         -3.93859856e-02, -4.94218469e-02, -1.26698771e-02,  1.48674166e-02,
         -2.49241143e-02,  1.23648476e-02, -1.09257374e-03,  1.64774656e-02,
         -2.71221772e-02,  4.72010439e-03, -5.62958792e-02, -2.48882845e-02,
         -4.66945618e-02, -5.26446812e-02,  2.15499531e-02, -6.31058216e-02,
         -2.03465614e-02, -4.50553671e-02, -3.89651917e-02, -1.24005629e-02,
          2.38795951e-03, -3.35833579e-02, -4.46390398e-02, -1.88498944e-02,
          2.13144515e-02, -1.10748708e-02, -3.63614783e-02,  1.75366662e-02,
          2.71381922e-02, -5.22868708e-02, -5.63306697e-02, -2.71824673e-02,
         -4.58585247e-02,  4.04285938e-02, -4.20641974e-02,  6.31884933e-02,
          1.91940367e-02, -1.57792214e-02,  8.48760009e-02, -1.87310576e-02,
         -5.18826908e-03, -5.45381308e-02, -5

In [119]:
import numpy as np

def df_to_embedding_arrays_efficient(df, image_prefix='image_url', clip_prefix='fashion_clip', n_images=11,
                                    save_embeddings_path='fashion_clip.npy', save_urls_path='image_urls.npy'):
    # Count total valid entries
    total_valid = sum(
        (df[f'{image_prefix}{i}'].notna() & df[f'{clip_prefix}{i}'].notna()).sum()
        for i in range(1, n_images + 1)
    )
    
    # Get embedding dimension from first non-NaN entry
    sample_clip = next(iter(df[f'{clip_prefix}1'].dropna()))
    embedding_dim = len(sample_clip)
    
    embeddings_array = np.zeros((total_valid, embedding_dim), dtype=np.float32)
    urls_array = np.empty(total_valid, dtype=object)
    
    idx = 0
    for i in range(1, n_images + 1):
        image_col = f'{image_prefix}{i}'
        clip_col = f'{clip_prefix}{i}'
        
        valid = df[image_col].notna() & df[clip_col].notna()
        
        for img, clip in zip(df.loc[valid, image_col], df.loc[valid, clip_col]):
            embeddings_array[idx] = np.array(clip, dtype=np.float32)
            urls_array[idx] = img
            idx += 1
    
    # Save arrays
    np.save(save_embeddings_path, embeddings_array)
    np.save(save_urls_path, urls_array)
    
    # Build URL → index map for fast lookup
    url_to_index = {url: i for i, url in enumerate(urls_array)}
    
    return embeddings_array, urls_array, url_to_index

# Usage
embeddings_array, urls_array, url_to_index = df_to_embedding_arrays_efficient(df_emb)

In [121]:
embeddings_array = np.load('data/embeddings/fashion_clip.npy', allow_pickle=True)

In [122]:
embeddings_array

array([[-0.05244665,  0.12732604, -0.02748385, ..., -0.00307378,
         0.00221065, -0.00060897],
       [ 0.03478447,  0.03196311, -0.00114067, ...,  0.00863611,
        -0.02320911, -0.01986619],
       [-0.01006423,  0.08902811, -0.11284529, ..., -0.00399827,
        -0.00726509, -0.02131907],
       ...,
       [ 0.0130894 ,  0.02811509, -0.02549931, ...,  0.0205684 ,
         0.01676122, -0.01086139],
       [-0.01131297,  0.00769202,  0.01837054, ..., -0.00583046,
         0.03069436,  0.00898907],
       [-0.01043592,  0.05363011, -0.02954775, ..., -0.01433267,
         0.01527151, -0.01945349]], shape=(136299, 512), dtype=float32)

In [118]:
embeddings_array

array([[-0.05244665,  0.12732604, -0.02748385, ..., -0.00307378,
         0.00221065, -0.00060897],
       [ 0.03478447,  0.03196311, -0.00114067, ...,  0.00863611,
        -0.02320911, -0.01986619],
       [-0.01006423,  0.08902811, -0.11284529, ..., -0.00399827,
        -0.00726509, -0.02131907],
       ...,
       [-0.00608524,  0.06292278, -0.02877053, ..., -0.02468918,
         0.00634032, -0.00579401],
       [-0.01044661,  0.07204853, -0.09876425, ..., -0.0305025 ,
        -0.02606086, -0.06021779],
       [-0.01990339,  0.11095314, -0.10203238, ..., -0.00174182,
        -0.02222033, -0.02873603]], shape=(136249, 512), dtype=float32)

In [111]:
url_of_interest = 'https://assets.vogue.com/photos/591b5547ecc6916578b6f84b/master/w_1280,c_limit/_SOR0298.jpg'
embedding = embeddings_array[url_to_index[url_of_interest]]

In [103]:
data_emb = np.load('/Users/traopia/Downloads/VogueRunway_image.npy', allow_pickle=True)  # allow_pickle=True is needed if dtype=object

In [106]:
data_emb_new = np.load('data/fashion_data_emb.npy', allow_pickle=True)  # allow_pickle=True is needed if dtype=object
print(type(data_emb_new))  # usually <class 'numpy.ndarray'>
print(data_emb_new.shape)  # number of rows
print(data_emb_new[:5])   

<class 'numpy.ndarray'>
(136249, 2)
[['https://assets.vogue.com/photos/591b5547ecc6916578b6f84b/master/w_1280,c_limit/_SOR0298.jpg'
  list([-0.052446648500000005, 0.12732604150000001, -0.0274838544, -0.0123123433, -0.0393859856, -0.049421846900000004, -0.0126698771, 0.0148674166, -0.0249241143, 0.012364847600000001, -0.0010925737000000001, 0.0164774656, -0.0271221772, 0.0047201044, -0.0562958792, -0.0248882845, -0.0466945618, -0.0526446812, 0.0215499531, -0.0631058216, -0.0203465614, -0.0450553671, -0.0389651917, -0.0124005629, 0.0023879595, -0.0335833579, -0.0446390398, -0.0188498944, 0.0213144515, -0.0110748708, -0.0363614783, 0.017536666200000002, 0.0271381922, -0.0522868708, -0.0563306697, -0.027182467300000002, -0.0458585247, 0.0404285938, -0.0420641974, 0.0631884933, 0.0191940367, -0.0157792214, 0.0848760009, -0.0187310576, -0.0051882691, -0.0545381308, -0.054924801, 0.029941052200000002, -0.005985709400000001, 0.012917925600000001, 0.0170597024, 0.0064474121, -0.0235349648000000

In [105]:
print(type(data_emb))  # usually <class 'numpy.ndarray'>
print(data_emb.shape)  # number of rows
print(data_emb[:5])   

<class 'numpy.ndarray'>
(1281633, 512)
[[ 0.01915    0.00354    0.005207  ...  0.0702    -0.02264    0.01811  ]
 [ 0.0202     0.004265  -0.0116    ...  0.0373     0.013466   0.0249   ]
 [ 0.0085    -0.002226  -0.01901   ...  0.05255   -0.01366    0.014725 ]
 [-0.03323    0.0152     0.01406   ...  0.02705   -0.01991   -0.002161 ]
 [-0.01935   -0.0018425  0.01842   ...  0.0187    -0.001188   0.0007577]]
